##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introdução ao TensorBoard

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tensorboard/get_started"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tensorboard/get_started.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tensorboard/get_started.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tensorboard/get_started.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

No aprendizado de máquina, para melhorar algo, você geralmente precisa conseguir medi-lo. O TensorBoard é uma ferramenta que fornece medições e visualizações necessárias durante o fluxo de trabalho do aprendizado de máquina. Ele permite monitorar métricas de experimento, como perda e exatidão, visualizar o grafo do modelo, projetar embeddings em um espaço de menos dimensões e muito mais.

Este guia rápido mostra como começar a usar o TensorBoard. Os outros guias deste site oferecem mais detalhes sobre recursos específicos, muitos deles não inclusos aqui. 

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import datetime

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

Usando o dataset [MNIST](https://en.wikipedia.org/wiki/MNIST_database) como exemplo, normalize os dados e escreva uma função que cria um modelo simples do Keras para classificar as imagens em 10 classes.

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='layers_flatten'),
    tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),
    tf.keras.layers.Dropout(0.2, name='layers_dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2')
  ])

11493376/11490434 [==============================] - 0s 0us/step


## Uso do TensorBoard com o Model.fit() do Keras

Ao treinar com o [Model.fit()](https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#fit) do Keras, a inclusão da callback `tf.keras.callbacks.TensorBoard` garante a criação e o armazenamento dos logs. Além disso, ative a computação do histograma de cada época com `histogram_freq=1` (é desativada por padrão).

Coloque os logs em um subdiretório com carimbos de data/hora para selecionar facilmente execuções de treinamento diferentes.

In [ ]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 15s 246us/sample - loss: 0.2217 - accuracy: 0.9343 - val_loss: 0.1019 - val_accuracy: 0.9685
Epoch 2/5
60000/60000 [==============================] - 14s 229us/sample - loss: 0.0975 - accuracy: 0.9698 - val_loss: 0.0787 - val_accuracy: 0.9758
Epoch 3/5
60000/60000 [==============================] - 14s 231us/sample - loss: 0.0718 - accuracy: 0.9771 - val_loss: 0.0698 - val_accuracy: 0.9781
Epoch 4/5
60000/60000 [==============================] - 14s 227us/sample - loss: 0.0540 - accuracy: 0.9820 - val_loss: 0.0685 - val_accuracy: 0.9795
Epoch 5/5
60000/60000 [==============================] - 14s 228us/sample - loss: 0.0433 - accuracy: 0.9862 - val_loss: 0.0623 - val_accuracy: 0.9823


Inicialize o TensorBoard pela linha de comando ou em uma experiência de notebook. As duas interfaces são geralmente iguais. Em notebooks, use o magic de linha `%tensorboard`. Na linha de comando, execute o mesmo comando sem o "%".

In [ ]:
%tensorboard --logdir logs/fit

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_model_fit.png?raw=1"/> -->

Uma breve visão geral das visualizações criadas nesse exemplo e dos painéis de controle (guias na barra de navegação na parte superior) em que elas podem ser encontradas:

- **Escalares** mostram como a perda e as métricas mudam a cada época. Você pode usá-los para monitorar a velocidade do treinamento, a taxa de aprendizado e outros valores escalares. Eles podem ser encontrados nos painéis de controle **Time Series** (Séries temporais) ou **Scalars** (Escalares).
- **Grafos** ajudam na visualização do seu modelo. Nesse caso, o grafo de camadas do Keras é exibido, para você garantir que ele foi criado corretamente. Os grafos podem ser encontrados no painel de controle **Graphs** (Grafos).
- **Histogramas** e **Distribuições** mostram a distribuição de um Tensor ao longo do tempo. Isso pode ser útil para visualizar pesos e biases e verificar se eles estão mudando de uma maneira esperada. Os histogramas podem ser encontrados nos painéis de controle **Time Series** ou **Histograms** (Histogramas). As distribuições podem ser encontradas no painel de controle **Distributions** (Distribuições).

Os painéis de controle do TensorBoard adicionais são ativados automaticamente quando você registra outros tipos de dados. Por exemplo, a callback do TensorBoard para o Keras deixa você registrar imagens e embeddings também. Você pode ver os outros painéis de controle disponíveis no TensorBoard ao clicar no menu suspenso "inativo" no canto superior direito.

## Uso do TensorBoard com outros métodos


Ao treinar com outros métodos, como [`tf.GradientTape()`](https://www.tensorflow.org/api_docs/python/tf/GradientTape), use `tf.summary` para registrar as informações necessárias.

Use o mesmo dataset acima, mas converta para `tf.data.Dataset`, aproveitando a funcionalidade de lote:

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

O código de treinamento segue o tutorial [guia rápido avançado](https://www.tensorflow.org/tutorials/quickstart/advanced), mas mostra como registrar as métricas para o TensorBoard. Escolha a perda e o otimizador:

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

Crie métricas stateful que possam ser usadas para acumular valores durante o treinamento e registradas a qualquer momento:

In [ ]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

Defina as funções de treinamento e teste:

In [ ]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

Configure escritores de resumo para escrever os resumos no disco em um diretório de logs diferente:

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

Inicie o treinamento. Use `tf.summary.scalar()` para registrar métricas (perda e exatidão) durante o treinamento/teste dentro do escopo dos escritores de resumo para escrever os resumos no disco. Você tem controle sobre quais métricas registrar e com que frequência. Outras funções `tf.summary` permitem o registro de outros tipos de dados.

In [ ]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.24321186542510986, Accuracy: 92.84333801269531, Test Loss: 0.13006582856178284, Test Accuracy: 95.9000015258789
Epoch 2, Loss: 0.10446818172931671, Accuracy: 96.84833526611328, Test Loss: 0.08867532759904861, Test Accuracy: 97.1199951171875
Epoch 3, Loss: 0.07096975296735764, Accuracy: 97.80166625976562, Test Loss: 0.07875105738639832, Test Accuracy: 97.48999786376953
Epoch 4, Loss: 0.05380449816584587, Accuracy: 98.34166717529297, Test Loss: 0.07712937891483307, Test Accuracy: 97.56999969482422
Epoch 5, Loss: 0.041443776339292526, Accuracy: 98.71833038330078, Test Loss: 0.07514958828687668, Test Accuracy: 97.5


Abra o TensorBoard novamente, desta vez apontando-o para um novo diretório de log. Também poderíamos iniciar o TensorBoard para monitorar o treinamento enquanto ele progride.

In [ ]:
%tensorboard --logdir logs/gradient_tape

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_gradient_tape.png?raw=1"/> -->

Pronto! Agora você já viu como usar o TensorBoard pela callback do Keras e pela `tf.summary` para cenários mais personalizados. 

## TensorBoard.dev: hospede e compartilhe os resultados dos seus experimentos de ML

[TensorBoard.dev](https://tensorboard.dev) é um serviço público e gratuito que permite fazer o upload dos seus logs do TensorBoard e obter um permalink que possa ser compartilhado com qualquer pessoa em artigos acadêmicos, postagens de blog, redes sociais etc. Isso pode promover melhor reprodutibilidade e colaboração.

Para usar o TensorBoard.dev, execute o seguinte comando:


In [ ]:
!tensorboard dev upload \
  --logdir logs/fit \
  --name "(optional) My latest experiment" \
  --description "(optional) Simple comparison of several hyperparameters" \
  --one_shot

Observe que essa invocação usa o prefixo de exclamação (`!`) para invocar o shell em vez do prefixo de porcentagem (`%`) para invocar o magic do colab.  Ao invocar esse comando a partir da linha de comando, não é necessário nenhum prefixo.

Veja um exemplo [aqui](https://tensorboard.dev/experiment/EDZb7XgKSBKo6Gznh3i8hg/#scalars).

Para mais detalhes sobre como usar o TensorBoard.dev, veja https://tensorboard.dev/#get-started